In [1]:
import pandas as pd

import numpy as np

from datetime import datetime

import matplotlib.pyplot as plt

import seaborn as sns

# LOAD DATA

df = pd.read_csv('data/online_retail_II.csv')

# Check structure

print("Data loaded successfully!")

print(f"Total rows: {len(df)}")

print(f"Total columns: {len(df.columns)}")

print("\nFirst 5 rows:")

print(df.head())


Data loaded successfully!
Total rows: 1067371
Total columns: 8

First 5 rows:
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  
0  2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3  2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4  2009-12-01 07:45:00   1.25      13085.0  United Kingdom  


In [4]:
# DATA CLEANING

print("=== CLEANING DATA ===\n")

# Remove missing Customer ID
df = df[df['Customer ID'].notna()]
print(f"Step 1: {len(df)} rows after removing null Customer ID")

# Remove negative quantities
df = df[df['Quantity'] > 0]
print(f"Step 2: {len(df)} rows after removing negative quantities")

# Remove invalid prices
df = df[df['Price'] > 0]
print(f"Step 3: {len(df)} rows after removing invalid prices")

# Create Revenue column
df['Revenue'] = df['Quantity'] * df['Price']

# Remove outliers
revenue_99th = df['Revenue'].quantile(0.99)
df = df[df['Revenue'] <= revenue_99th]
print(f"Step 4: {len(df)} rows after removing outliers")

# Convert date
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print(f"\n✓ Final dataset: {len(df)} rows")
print(f"✓ Unique customers: {len(df['Customer ID'].unique())}")




=== CLEANING DATA ===

Step 1: 824364 rows after removing null Customer ID
Step 2: 805620 rows after removing negative quantities
Step 3: 805549 rows after removing invalid prices
Step 4: 797520 rows after removing outliers

✓ Final dataset: 797520 rows
✓ Unique customers: 5816


In [7]:
# CALCULATE RFM METRICS

print("\n=== CALCULATING RFM ===\n")

# Reference date
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"Reference date: {reference_date}\n")

# Create RFM dataframe
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'Invoice': 'nunique',
    'Revenue': 'sum'
}).rename(columns={
    'InvoiceDate': 'Recency',
    'Invoice': 'Frequency',
    'Revenue': 'Monetary'
})

print("RFM Statistics:")
print(rfm.describe().round(2))



=== CALCULATING RFM ===

Reference date: 2011-12-10 12:50:00

RFM Statistics:
       Recency  Frequency   Monetary
count  5816.00    5816.00    5816.00
mean    201.38       6.17    2347.53
std     209.48      12.45    7384.42
min       1.00       1.00       2.90
25%      26.00       1.00     335.48
50%      95.00       3.00     847.58
75%     380.00       7.00    2139.30
max     739.00     381.00  267159.80


In [10]:
# CREATE RFM SCORES

print("\n=== RFM SCORING ===\n")

# Recency: Lower is better (reverse score)

rfm['R_Score'] = pd.qcut(rfm['Recency'], q=4, labels=[4, 3, 2, 1], duplicates='drop')

# Frequency: Higher is better

rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=4, labels=[1, 2, 3, 4], duplicates='drop')

# Monetary: Higher is better

rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=4, labels=[1, 2, 3, 4], duplicates='drop')

# Convert to numeric

rfm['R_Score'] = rfm['R_Score'].astype(int)

rfm['F_Score'] = rfm['F_Score'].astype(int)

rfm['M_Score'] = rfm['M_Score'].astype(int)

# Combined Score

rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

print("RFM Scores calculated!")

print("\nSample scores:")

print(rfm[['Recency', 'Frequency', 'Monetary', 'RFM_Score']].head(10))



=== RFM SCORING ===

RFM Scores calculated!

Sample scores:
             Recency  Frequency  Monetary RFM_Score
Customer ID                                        
12346.0          529         11    372.86       142
12347.0            2          8   5383.72       444
12348.0           75          5   1779.40       333
12349.0           19          4   3878.69       434
12350.0          310          1    334.40       211
12351.0          375          1    300.93       211
12352.0           36         10   2193.29       344
12353.0          204          2    406.76       222
12354.0          232          1   1079.40       213
12355.0          214          2    947.61       223


In [11]:
# SEGMENT CUSTOMERS

print("\n=== SEGMENTATION ===\n")

def segment_customer(rfm_score):
    r = int(rfm_score[0])
    f = int(rfm_score[1])
    m = int(rfm_score[2])
    
    if (r >= 3 and f >= 3 and m >= 3):
        return 'Champions'
    elif (r >= 2 and f >= 3 and m >= 3):
        return 'Loyal Customers'
    elif (r >= 3 and f >= 3):
        return 'Potential Loyalists'
    elif (r <= 2 and f >= 2) or (m >= 2 and r == 1):
        return 'At Risk'
    else:
        return 'Lost Customers'

rfm['Segment'] = rfm['RFM_Score'].apply(segment_customer)

print("Customer Distribution:")
print(rfm['Segment'].value_counts())

print("\nSegment Details:")
segment_summary = rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'sum', 'count']
}).round(2)

print(segment_summary)



=== SEGMENTATION ===

Customer Distribution:
Segment
Champions              1805
Lost Customers         1803
At Risk                1488
Loyal Customers         493
Potential Loyalists     227
Name: count, dtype: int64

Segment Details:
                    Recency Frequency Monetary                   
                       mean      mean     mean          sum count
Segment                                                          
At Risk              388.08      2.68   773.63   1151166.84  1488
Champions             28.98     13.71  5655.87  10208843.79  1805
Lost Customers       238.46      1.37   427.90    771501.89  1803
Loyal Customers      210.09      7.61  2815.57   1388075.25   493
Potential Loyalists   35.06      4.09   588.66    133626.90   227


In [12]:
# EXPORT FOR TABLEAU

print("\n=== EXPORTING DATA ===\n")

# Create outputs folder if needed

import os

os.makedirs('outputs', exist_ok=True)

# Export main RFM data

rfm_export = rfm.reset_index()

rfm_export.to_csv('outputs/rfm_segments.csv', index=False)

print("✓ Exported: outputs/rfm_segments.csv")

# Export segment summary

segment_summary = rfm.groupby('Segment').agg({

    'Recency': ['mean', 'min', 'max'],

    'Frequency': ['mean', 'min', 'max'],

    'Monetary': ['mean', 'min', 'max', 'sum', 'count']

}).round(2)

segment_summary.to_csv('outputs/segment_summary.csv')

print("✓ Exported: outputs/segment_summary.csv")

print("\n✓✓✓ PROJECT 1 COMPLETE! ✓✓✓")

print("\nNext: Upload rfm_segments.csv to Tableau")




=== EXPORTING DATA ===

✓ Exported: outputs/rfm_segments.csv
✓ Exported: outputs/segment_summary.csv

✓✓✓ PROJECT 1 COMPLETE! ✓✓✓

Next: Upload rfm_segments.csv to Tableau
